<a href="https://colab.research.google.com/github/d9249/DACON/blob/main/Producing_InceptionResNetV2_1_(public_%2C_private_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Oct  4 14:24:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터 보기
import pandas as pd
import numpy as np
from glob import glob

# 이미지데이터 로딩
from PIL import Image
import cv2
from tqdm import tqdm

# 파일경로 설정
import os
import shutil
import json

# Modeling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# GPU 환경 설정
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="3"

# Others
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [ ]:
model_save = 'InceptionResNetV2_1'
Target_model = 'InceptionResNetV2_model'
Target_predict = 'InceptionResNetV2_predict'
Target_acc = 'InceptionResNetV2_acc'
Target_val = 'InceptionResNetV2_val'

In [4]:
!mkdir open_data
!unzip -uq "/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/open.zip" -d "/content/open_data/"

In [ ]:
import pandas as pd
data_path = '/content/open_data/open'

train_path = data_path + '/train'
test_path = data_path + '/test'

hand_gesture = pd.read_csv(data_path + '/hand_gesture_pose.csv')
sample_submission = pd.read_csv(data_path + '/sample_submission.csv')

In [ ]:
answers = []
for train_folder in train_folders :
    json_path = glob(train_folder + '/*.json')[0]
    js = json.load(open(json_path))
    cat = js.get('annotations')[0].get('category_id')
    cat_name = js.get('annotations')[0].get('category_name')
    answers.append([train_folder.replace(data_path,''),cat, cat_name])

answers = pd.DataFrame(answers, columns = ['train_path','answer', 'answer_name'])
answers

In [ ]:
classes = pd.get_dummies(answers[['answer']], columns = ['answer']).to_numpy()

In [ ]:
import tensorflow as tf

Target_model =  tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes=classes)

In [ ]:
Target_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator (
    rescale = 1./255, 
    validation_split = 0.075,
    rotation_range = 15,
    width_shift_range = 0.00,
    height_shift_range = 0.05 )

batch_size = 8
train_generator = datagen.flow_from_directory(train_path, target_size=(224,224,3), batch_size = batch_size, color_mode='rgb', class_mode='categorical', subset='training')
val_generator = datagen.flow_from_directory(train_path, target_size=(224,224,3), batch_size = batch_size, color_mode='rgb', class_mode='categorical', subset='validation')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/Checkpoint/'+ model_save +'.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
Target_model.fit_generator(train_generator, epochs = 10, validation_data=val_generator, callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(Target_model.history.history["accuracy"], label = Target_acc)
plt.plot(Target_model.history.history["val_accuracy"], label = Target_val)

plt.legend()
plt.show()

In [ ]:
Target_model = tf.keras.models.load_model('/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/Checkpoint/'+ model_save +'.h5', compile=False)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory(test_path, target_size=(224,224), color_mode='rgb', class_mode='categorical', shuffle=False)

In [ ]:
Target_predict = Target_model.predict_generator(test_generator).argmax(axis=1)

In [ ]:
submission = pd.read_csv('/content/open_data/open/sample_submission.csv')

In [ ]:
predictions = []
for test_image in tqdm(test_images, total = len(test_images)) : 
    prediction = np.mean(model.predict(np.array(test_image)), axis = 0)
    predictions.append(prediction)

sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv('./BASELINE.csv', index=False)

In [ ]:
import numpy as np
mylist = []

for i in range(len(submission)):
    name =  test_generator.filenames
    id = name[i].split('/')[1].rstrip('.').split('.')[0]
    mylist.append(id)

In [ ]:
for i in range(len(submission)):
    submission["id"][i] = mylist[i]

In [ ]:
submission["model_predict"] = Target_predict

In [ ]:
from collections import Counter

for i in range(len(submission)) :
    predicts = submission.loc[i, ['model_predict']]
    submission.at[i, "digit"] = Counter(predicts).most_common(n=1)[0][0]

In [ ]:
submission = submission[['id', 'digit']]

In [ ]:
from google.colab import files

submission.to_csv('/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/submission/'+ model_save +'.csv', index=False)
files.download('/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/submission/'+ model_save +'.csv')